# **AFL Agent Tutorial: Notebook 03** <a href="https://colab.research.google.com/github/martintb/AFL-tutorial/blob/main/notebooks/03-afl-dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The AFL Agent Codebase

# Development Version

*Author: Tyler Martin - tyler.martin@nist.gov*


## Learning Goals

This notebook will introduce the following concepts:

- how to use the AFL.double_agent codebase to build decision pipelines


# Setup

**This may take several minutes to complete, please run ASAP**

Please run this collapsed cell by clicking in the box to the left of *8 cells hidden*. This will `pip install` and `import` all necessary dependencies for this tutorial. There is not need to expand this cell block, but you can if you're interested in looking under the hood.


If you've never used Google Colab before, [this FAQ](https://research.google.com/colaboratory/faq.html) may be useful to skim. It's very similar (and based off of) Jupyter, but divergent in several ways.

**Warning!**

If you disconnect for too long, you'll have to rerun this cell. Any text you type into this notebooks should always be saved, but variables in memory and files can dissappear when you disconnect.

You're free to download this notebook and run it locally if you're inclined, but we will provide no support for this.

In [ ]:
!git clone https://github.com/martintb/AFL-tutorial

In [ ]:
#!pip install --find-links ./AFL-tutorial/wheels --editable ./AFL-tutorial/

!pip install git+https://github.com/usnistgov/AFL-agent

In [ ]:
import sys
sys.path.insert(0,'./AFL-tutorial')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
import os

import sklearn.cluster
from sklearn.metrics import pairwise
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import Matern

import plotly.express as px
import plotly.graph_objects as go

import xarray as xr
xr.set_options(display_expand_data=False)

In [ ]:
np.random.seed(240424)

# Dev